In [185]:
import requests
from bs4 import BeautifulSoup
import csv
import json
URL = "https://baraasalout.github.io/test.html"
page = requests.get(URL) #get page
# print(page.text)

In [186]:
soup = BeautifulSoup(page.content, "html.parser") #parse the page

In [187]:
x=[]#to have all elements with their values
#get heading 1s
for h in soup.find_all("h1"):
    x.append({"Heading": "Heading1", "Content": h.get_text().strip()})
#get heading 2s
for h in soup.find_all("h2"):
    x.append({"Heading": "Heading2", "Content": h.get_text().strip()})
#get paragraphs
for h in soup.find_all("p"):
    x.append({"Heading": "Paragraph", "Content": h.get_text().strip()})
#get listitems
for h in soup.find_all("li"):
    x.append({"Heading": "Listitem", "Content": h.get_text().strip()})
#save as csv 
with open("Practice.csv", mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=["Heading", "Content"])
    writer.writeheader()
    writer.writerows(x)

print(x)

[{'Heading': 'Heading1', 'Content': 'Web Scraping Practice'}, {'Heading': 'Heading2', 'Content': 'Available Products'}, {'Heading': 'Heading2', 'Content': 'Product Table'}, {'Heading': 'Heading2', 'Content': 'Watch This Video'}, {'Heading': 'Heading2', 'Content': 'Contact Us'}, {'Heading': 'Heading2', 'Content': 'Product Information'}, {'Heading': 'Heading2', 'Content': 'Featured Products'}, {'Heading': 'Paragraph', 'Content': 'Welcome to the web scraping task! Use your skills to extract the required data from this page.'}, {'Heading': 'Paragraph', 'Content': 'Sharp Objects'}, {'Heading': 'Paragraph', 'Content': '£47.82'}, {'Heading': 'Paragraph', 'Content': '✔ In stock'}, {'Heading': 'Paragraph', 'Content': 'In a Dark, Dark Wood'}, {'Heading': 'Paragraph', 'Content': '£19.63'}, {'Heading': 'Paragraph', 'Content': '✔ In stock'}, {'Heading': 'Paragraph', 'Content': 'The Past Never Ends'}, {'Heading': 'Paragraph', 'Content': '£56.50'}, {'Heading': 'Paragraph', 'Content': '✔ In stock'}, {

In [188]:
table = [i.get_text() for i in soup.find_all("th")]#get the headers
td=[i.get_text() for i in soup.find_all("td")]#get values
print(table)
print(td)

['Product', 'Price', 'In Stock']
['Laptop', '$1000', 'Yes', 'Smartphone', '$800', 'No', 'Tablet', '$500', 'Yes']


In [189]:
product=[]
price=[]
instock=[]
for i in range(0,len(td),3):
    product.append(td[i])#take the first as product name 
    price.append(td[i+1])#second as price
    instock.append(td[i+2])#third as instock or not 

x = []
for i in range(len(product)):#making them in a list of dicts
    x.append({
        "Product": product[i],
        "price": price[i],
        "instock": instock[i]
    })
# save as csv file
with open("Prods.csv", mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=["Product", "price","instock"])
    writer.writeheader()
    writer.writerows(x)
print(product)
print(price)
print(instock)

['Laptop', 'Smartphone', 'Tablet']
['$1000', '$800', '$500']
['Yes', 'No', 'Yes']


In [198]:
divs = soup.find_all("div",style="text-align: center; width: 200px; border: 1px solid #ddd; padding: 10px; border-radius: 5px;")#get the div with this characteristics
Book=[]
price=[]
stock=[]
buttons=[]
#getting book name price and stock or not as ordered in list 
for product in divs:
    paragraphs = product.find_all("p")
    button=product.find_all("button")
    if len(paragraphs) == 3:
        Book.append(paragraphs[0].get_text().strip())
        price.append(paragraphs[1].get_text().strip())
        #stock.append(paragraphs[2].get_text().strip())#if needed for just to stay the same 
        # print(paragraphs[2].get_text())
        if paragraphs[2].get_text().strip().endswith("In stock"):
            stock.append("Yes")
        else:
            stock.append("No")
        buttons.append([i.get_text() for i in button][0])
print(Book)
print(price)
print(buttons)
print(stock)
x = []
for i in range(len(Book)):#saving them in a list of dicts
    x.append({
        "Book": Book[i],
        "Price": price[i],
        "instock": stock[i],
        "button":buttons[i]
    })
with open('Books.json', 'w') as f:
    json.dump(x, f)

['Sharp Objects', 'In a Dark, Dark Wood', 'The Past Never Ends', 'A Murder in Time']
['£47.82', '£19.63', '£56.50', '£16.64']
['Add to basket', 'Add to basket', 'Add to basket', 'Add to basket']
['Yes', 'Yes', 'Yes', 'No']


In [191]:
fields = []
#get fields names ,types and defaults 
for tag in soup.find_all(['input', 'select', 'textarea']):
    field = {}
    name = tag.get('name')#name
    if name:
        field['name'] = name

    if tag.name == 'input':
        field['type'] = tag.get('type')#type
        if field['type'] in ['checkbox']: #default
            field['default'] = tag.has_attr('checked')#if it it is checked
        else:
            value = tag.get('value')#to know the value that is default
            if value:
                field['default'] = value
    elif tag.name == 'select':
        field['type'] = 'select'
        options = [option.get('value', '') for option in tag.find_all('option')]#options for select
        field['options'] = options

    if field:
        fields.append(field)

print(fields)
with open('fields.json', 'w') as f:
    json.dump(fields, f)

[{'name': 'username', 'type': 'text'}, {'name': 'password', 'type': 'password'}, {'name': 'options', 'type': 'select', 'options': ['option1', 'option2', 'option3']}, {'name': 'terms', 'type': 'checkbox', 'default': False}, {'type': 'submit', 'default': 'Submit'}]


In [192]:
links=[]
for tag in soup.find_all('a'):#get all a link text and url
    link_text = tag.get_text.strip()#linktext
    href = tag.get('href')#url
    link={link_text:href}#dict
    links.append(link)
print("links",links)
frame = soup.find_all("iframe")#get iframe for the video
link=[i.get("src") for i in frame]#get links
linkframe={"videolink":link}#put in dict
links.append(linkframe)
print("links with iframe",links)
with open('links.json', 'w') as f:
    json.dump(links, f)

links []
links with iframe [{'videolink': ['https://www.youtube.com/watch?v=ujf9RNuBdCU']}]


In [193]:
prodcards = soup.find_all("div",class_="product-card")#get all classes as product cards 
cards=[]
for prod in prodcards:#for each one we will have name price and colors and id of the whole product
    card={}
    card['id'] = prod.get('data-id')#id
    paragraphs = prod.find_all("p")#all paragraphs as their order no change
    if len(paragraphs) == 3:
        card['name']=paragraphs[0].get_text().strip()#first in list is name
        card['price']=paragraphs[1].get_text().strip()# second is price
        card['colors']=paragraphs[2].get_text().split(":")[1].strip() #third is colors
    cards.append(card)
print(cards)
with open('cards.json', 'w') as f:
    json.dump(cards, f)

[{'id': '101', 'name': 'Wireless Headphones', 'price': '$49.99', 'colors': 'Black, White, Blue'}, {'id': '102', 'name': 'Smart Speaker', 'price': '$89.99', 'colors': 'Grey, Black'}, {'id': '103', 'name': 'Smart Watch', 'price': '$149.99', 'colors': 'Black, Silver, Gold'}]
